## (https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html)

Basics of PyTorch Model Authoring

In [2]:
import torch
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.7605, 0.6402, 0.8270, 0.7194],
        [0.6587, 0.6450, 0.8486, 0.8972],
        [0.8298, 0.7481, 0.4894, 0.8096]]), tensor([[0.7605, 0.6402, 0.8270, 0.7194],
        [0.6587, 0.6450, 0.8486, 0.8972],
        [0.8298, 0.7481, 0.4894, 0.8096]]))


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[0.3883, 0.5426, 0.4675, 0.6050],
        [0.4570, 0.9007, 0.1697, 0.3920],
        [0.3178, 0.7466, 0.6151, 0.4564]], grad_fn=<TanhBackward0>), tensor([[0.3883, 0.5426, 0.4675, 0.6050],
        [0.4570, 0.9007, 0.1697, 0.3920],
        [0.3178, 0.7466, 0.6151, 0.4564]], grad_fn=<TanhBackward0>))


https://chat.openai.com/c/1818ab80-f73c-466d-94d3-a4fc11bc4d5e

In [4]:
#Using decision gate
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))


MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.7337, -0.2989,  0.8422,  0.5011],
        [ 0.4047,  0.0665,  0.7835,  0.3452],
        [ 0.7151,  0.1713,  0.8284,  0.4572]], grad_fn=<TanhBackward0>), tensor([[ 0.7337, -0.2989,  0.8422,  0.5011],
        [ 0.4047,  0.0665,  0.7835,  0.3452],
        [ 0.7151,  0.1713,  0.8284,  0.4572]], grad_fn=<TanhBackward0>))


## Tracing Modules

In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.6611,  0.6983,  0.4821,  0.8922],
         [ 0.6803,  0.3864,  0.6877,  0.8332],
         [-0.0039,  0.7190, -0.1046,  0.6200]], grad_fn=<TanhBackward0>),
 tensor([[ 0.6611,  0.6983,  0.4821,  0.8922],
         [ 0.6803,  0.3864,  0.6877,  0.8332],
         [-0.0039,  0.7190, -0.1046,  0.6200]], grad_fn=<TanhBackward0>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %linear : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # C:\Users\Himani Munjal\AppData\Local\Temp\ipykernel_31440\260609686.py:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%20, %h, %11) # C:\Users\Himani Munjal\AppData\Local\Temp\ipykernel_31440\260609686.py:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%12) # C:\Users\Himani Munjal\AppData\Local\Temp\ipykernel_31440\260609686.py:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%13, %13)
  return (%14)



In [7]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)



In [8]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.6611,  0.6983,  0.4821,  0.8922],
        [ 0.6803,  0.3864,  0.6877,  0.8332],
        [-0.0039,  0.7190, -0.1046,  0.6200]], grad_fn=<TanhBackward0>), tensor([[ 0.6611,  0.6983,  0.4821,  0.8922],
        [ 0.6803,  0.3864,  0.6877,  0.8332],
        [-0.0039,  0.7190, -0.1046,  0.6200]], grad_fn=<TanhBackward0>))
(tensor([[ 0.6611,  0.6983,  0.4821,  0.8922],
        [ 0.6803,  0.3864,  0.6877,  0.8332],
        [-0.0039,  0.7190, -0.1046,  0.6200]], grad_fn=<TanhBackward0>), tensor([[ 0.6611,  0.6983,  0.4821,  0.8922],
        [ 0.6803,  0.3864,  0.6877,  0.8332],
        [-0.0039,  0.7190, -0.1046,  0.6200]], grad_fn=<TanhBackward0>))


## Using Scripting to Convert Modules

In [9]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



C:\Users\Himani Munjal\AppData\Local\Temp\ipykernel_31440\4234398751.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [10]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)


def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



In [11]:
# New inputs
x, h = torch.rand(3, 4), torch.rand(3, 4)
print(scripted_cell(x, h))

(tensor([[-0.0150,  0.2681,  0.2044,  0.7610],
        [-0.2219,  0.3348,  0.4733,  0.4036],
        [ 0.5563,  0.6644,  0.4706,  0.4633]], grad_fn=<TanhBackward0>), tensor([[-0.0150,  0.2681,  0.2044,  0.7610],
        [-0.2219,  0.3348,  0.4733,  0.4036],
        [ 0.5563,  0.6644,  0.4706,  0.4633]], grad_fn=<TanhBackward0>))


## Mixing Scripting and Tracing


In [14]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4])
  y = torch.zeros([3, 4])
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    cell = self.cell
    _0 = (cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [15]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)



## Saving and Loading models

In [16]:
traced.save('wrapped_rnn.pt')

loaded = torch.jit.load('wrapped_rnn.pt')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)

